In [95]:
!pip3 install mesa

In [96]:
# La clase `Model` se hace cargo de los atributos a nivel del modelo, maneja los agentes. 
# Cada modelo puede contener múltiples agentes y todos ellos son instancias de la clase `Agent`.
from mesa import Agent, Model 

# Con `SimultaneousActivation` hacemos que todos los agentes se activen de manera simultanea.
from mesa.time import SimultaneousActivation

# Vamos a hacer uso de `DataCollector` para obtener el grid completo cada paso (o generación) y lo usaremos para graficarlo.
from mesa.datacollection import DataCollector

# mathplotlib lo usamos para graficar/visualizar como evoluciona el autómata celular.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Definimos los siguientes paquetes para manejar valores númericos.
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime

TOTAL_MOVES  = 0
boxesOnShelf = 0
shelfX       = 0
shelfY       = 0
#estantepos = [0, 0]

def get_grid(model):
    '''
    Esta es una función auxiliar que nos permite guardar el grid para cada uno de los agentes.
    param model: El modelo del cual optener el grid.
    return una matriz con la información del grid del agente.
    '''
    grid = np.zeros((model.grid.width, model.grid.height))
    for cell in model.grid.coord_iter():
        cell_content, x, y = cell
        for content in cell_content:
            if isinstance(content, Robot):
                grid[x][y] = 7
            elif isinstance(content, Box):
                if content.live >= 1:
                    grid[x][y] = content.numBoxes
                else:
                    grid[x][y] = 0
            #else:
            #    grid[x][y] = 0
    return grid

In [97]:
class RandomWalker(Agent):

    grid = None
    x = None
    y = None
    moore = True
    

    def __init__(self, unique_id, pos, model, moore=False):
        super().__init__(unique_id, model)
        self.pos = pos
        self.moore = moore

    def random_move(self):
        # Pick the next cell from the adjacent cells.
        #next_moves = self.model.grid.get_neighborhood(self.pos, self.moore, include_center = False)
        #print(next_moves)
        #next_move = self.random.choice(next_moves)

        neighbors  = self.model.grid.get_neighbors(self.pos, self.moore, include_center = False)
        aux = [obj for obj in neighbors if not isinstance(obj, Robot)]
        #print("neighbors")
        #print(neighbors)
        #print("aux")
        #print(aux)

        # Now move:
        if len(aux) > 0:
          next_move = self.random.choice(aux)
          self.model.grid.move_agent(self, next_move.pos)
        else:
          self.model.grid.move_agent(self, self.pos)


    def move_to_shelf(self):
        global shelfX
        global shelfY
        estantepos=(shelfX, shelfY)
        #muevete al estante
        next_move = self.pos

        #estantepos=(shelfX,shelfY)
        if next_move[0] > estantepos[0]: #Si el estante esta hacia arriba
          next_move = (next_move[0]-1,next_move[1])
        elif next_move[0] < estantepos[0]:
          next_move = (next_move[0]+1,next_move[1])
        elif next_move[1] > estantepos[1]: #Si el estante esta hacia la izquierda
          next_move = (next_move[0],next_move[1]-1)
        elif next_move[1] < estantepos[1]: #Si el estante esta hacia la derecha
          next_move = (next_move[0],next_move[1]+1)
        

        neighbors  = self.model.grid.get_neighbors(self.pos, self.moore, include_center = False)
        aux = [obj for obj in neighbors if not isinstance(obj, Robot)]

        if len(aux) > 0:
          self.model.grid.move_agent(self, next_move)
        else:
          self.model.grid.move_agent(self, self.pos)
        #self.model.grid.move_agent(self, next_move)
        

        


In [98]:
class Robot(RandomWalker):
    def __init__(self, unique_id, pos, model, moore):
        super().__init__(unique_id, pos, model, moore=moore)
        self.live=7
        self.load=False
        #self.moves=0

    def step(self):
        global shelfX
        global shelfY
        global TOTAL_MOVES
        global boxesOnShelf
        estantepos = (shelfX, shelfY)

        if self.load == False:
          self.random_move()
          TOTAL_MOVES = TOTAL_MOVES + 1
                  
          #Si hay caja, agarra caja
          #x, y = self.pos
          this_cell = self.model.grid.get_cell_list_contents([self.pos])
          boxes = [obj for obj in this_cell if isinstance(obj, Box) and obj.numBoxes == 1]
          if len(boxes) > 0 and self.pos != estantepos:
              boxes_to_load = self.random.choice(boxes)

              # Elimina suciedad
              boxes_to_load.live = 0
              boxes_to_load.numBoxes = 0
              #boxes_to_load.pos = estantepos
              self.load = True
              #print(sheep_to_load.live)
        else:
          #muevete al estante incompleto
          if self.pos != estantepos: 
            self.move_to_shelf();
            TOTAL_MOVES = TOTAL_MOVES + 1
          else:
            #checa si esta completo

            #coloca caja
            if self.pos == estantepos:
              this_cell = self.model.grid.get_cell_list_contents([self.pos])
              boxes = [obj for obj in this_cell if isinstance(obj, Box)] #>=1
              #print(boxes)
              if boxes[0].numBoxes > 4: #Cuando ya estoy lleno, haz esto...
                if estantepos[1] + 1 < width:
                  #global shelfY
                  shelfX = shelfX + 1
                  #next_move = (next_move[0],next_move[1]+1)
                  #self.model.grid.move_agent(self, next_move)
                elif estantepos[0] + 1 < height:
                  #global shelfX
                  shelfY = shelfY + 1
                  #next_move = (next_move[0]+1,next_move[1])
                  #self.model.grid.move_agent(self, next_move)
              if boxes[0].numBoxes < 5: #Solo para verificar que exista algo de tipo piso ahi
                new_box = boxes[len(boxes)-1]
                new_box.live=1;
                new_box.numBoxes+=1;
              boxesOnShelf+=1;

              self.load=False
                
class Box(Agent):
    '''
    Representa a un agente o una celda con estado vivo (1) o muerto (0)
    '''
    def __init__(self, unique_id, pos, live, numBoxes, model):
        '''
        Crea un agente con estado inicial aleatorio de 0 o 1, también se le asigna un identificador 
        formado por una tupla (x,y). También se define un nuevo estado cuyo valor será definido por las 
        reglas mencionadas arriba.
        '''
        super().__init__(unique_id, model)
        self.live = live
        self.numBoxes = numBoxes;
        self.pos=pos
        self.next_state = None
    
    def step(self):
        this_cell = self.model.grid.get_cell_list_contents([self.pos])

In [99]:
from mesa import Model
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector


class CleaningModel(Model):
    def __init__(self, width, height, initial_boxes, initial_bot):
        #self.num_agents = width * height
        self.initial_boxes=initial_boxes
        self.initial_bot=initial_bot
        self.grid = MultiGrid(width, height, False)
        self.schedule = SimultaneousActivation(self)
        
        
        
        for i in range(width):
          for j in range(height):
            boxes = Box(i*height+j,(i, j), 0, 0, self)
            self.grid.place_agent(boxes, (i, j))
            self.schedule.add(boxes)

        #spawn CAJAS
        for i in range(self.initial_boxes):
            x = self.random.randrange(width)
            y = self.random.randrange(height)

            this_cell = self.grid.get_cell_list_contents([(x, y)])
            boxes = [obj for obj in this_cell if isinstance(obj, Box) and obj.numBoxes >= 1]

            while len(boxes) >= 1 or x == 0 and y == 0:
              x = self.random.randrange(width)
              y = self.random.randrange(height)

              this_cell = self.grid.get_cell_list_contents([(x, y)])
              boxes = [obj for obj in this_cell if isinstance(obj, Box) and obj.numBoxes >= 1]
              
            if len(boxes) <= 0:
              boxes = Box(i+(width*height),(x, y), 1, 1, self)
              self.grid.place_agent(boxes, (x, y))
              self.schedule.add(boxes)

        #spawn aspiradoras
        for i in range(self.initial_bot):
          x = self.random.randrange(width)
          y = self.random.randrange(height)

          this_cell = self.grid.get_cell_list_contents([(x, y)])
          boxes = [obj for obj in this_cell if isinstance(obj, Box) and obj.numBoxes >= 1]
          bots  = [obj for obj in this_cell if isinstance(obj, Robot)]

          while len(boxes) >= 1 or len(bots) >= 1 or x == 0 and y == 0:
            x = self.random.randrange(width)
            y = self.random.randrange(height)

            this_cell = self.grid.get_cell_list_contents([(x, y)])
            boxes = [obj for obj in this_cell if isinstance(obj, Box) and obj.numBoxes >= 1]
            bots  = [obj for obj in this_cell if isinstance(obj, Robot)]
            
          if len(boxes) <= 0 and len(bots) <= 0:
            bot = Robot(i+initial_boxes+(width*height), (x, y), self, False)
            self.grid.place_agent(bot, (x, y))
            self.schedule.add(bot)
        
        # Aquí definimos con colector para obtener el grid completo.
        self.datacollector = DataCollector(
            model_reporters={"Grid": get_grid})
        
    def step(self):
        self.schedule.step()
        self.datacollector.collect(self)
        

In [100]:
#Datos Iniciales
TIEMPO_MAXIMO_EJECUCION = 1.0 #Tiempo Maximo de Ejecucion
initial_bots            = 5 #Numero de Agentes
initial_boxes           = 20
width                   = 10 #M
height                  = 10 #N
NUM_GENERATIONS         = 0

#cont=0
start_time = time.time()
tiempo_inicio=str(datetime.timedelta(seconds=TIEMPO_MAXIMO_EJECUCION))
model = CleaningModel(width, height, initial_boxes, initial_bots)
while((time.time()-start_time) < TIEMPO_MAXIMO_EJECUCION and NUM_GENERATIONS <= 250 and boxesOnShelf < initial_boxes):
    #print(cont)
    model.step()
    NUM_GENERATIONS += 1

# Imprimimos el tiempo que le tomó correr al modelo.
print('Tiempo de ejecución:', str(datetime.timedelta(seconds=(time.time() - start_time))))



Tiempo de ejecución: 0:00:00.155673


In [101]:
all_grid = model.datacollector.get_model_vars_dataframe()
print(type(all_grid))
print(all_grid)

<class 'pandas.core.frame.DataFrame'>
                                                  Grid
0    [[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0,...
1    [[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0,...
2    [[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0,...
3    [[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0,...
4    [[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0,...
..                                                 ...
246  [[5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
247  [[5.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
248  [[5.0, 0.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
249  [[5.0, 0.0, 0.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
250  [[5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...

[251 rows x 1 columns]


In [102]:
%%capture
fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(all_grid.iloc[0][0], cmap='Greys')

def animate(i):
    patch.set_data(all_grid.iloc[i][0])
    
anim = animation.FuncAnimation(fig, animate, frames=len(all_grid))

In [103]:
anim

In [104]:
TOTAL_MOVES

1238

In [105]:
if NUM_GENERATIONS > 250:
  print("Llego al limite de generaciones.")
else:
  print("Atraparon todas las cajas!")

print(" ")
print("Numero de generaciones: ", NUM_GENERATIONS-1)
print("Numero de cajas colocadas correctamente: " + str(boxesOnShelf) + "/" + str(initial_boxes));

Llego al limite de generaciones.
 
Numero de generaciones:  250
Numero de cajas colocadas correctamente: 17/20
